In [1]:
import gdal
import numpy as np
import glob
import os
import zipfile
import osr
from gdalconst import GA_ReadOnly
import datetime

In [5]:
# Download NLDAS SM Noah data using wget: generatet the file list

year = np.arange(2022, 2023, 1)
julian = np.arange(1,366,1)
url = 'https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_NOAH0125_H.002/'
name = 'NLDAS_NOAH0125_H.A'
ext = '.002.grb'
f = open('R:/UnitedStates/ForDRI_project-NEW/Case_Study_2022_Growing_Season/SM/urls.txt','w')
for y in year:
    url_y = url+str(year)+'/'
    for j in julian:
        if j<10:
            jjj = '00'+str(j)
        elif j<100:
            jjj = '0'+str(j)
        else:
            jjj = str(j)
        yyyymmdd = datetime.datetime.strptime(str(y)+jjj,'%Y%j').date().strftime('%Y%m%d')
        for h in range(0,24):
            if h < 10:
                hhhh = '0'+str(h)+'00'
            else:
                hhhh = str(h)+'00'
            full_url = url+str(y)+'/'+jjj+'/'+name+yyyymmdd+'.'+hhhh+ext
            f.write(full_url+'\n')
f.close()

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from requests.auth import HTTPBasicAuth
import re
import urllib.request
import requests

def getFile(url):
    try:
        html =urlopen(url)
    except HTTPError as e:
        return None
    try:
        
        bsObj = BeautifulSoup(html)
        files = bsObj.findAll("a")
        flist = []
        for link in files:
            if 'href' in link.attrs:
                flist.append(link.attrs['href'])
                #print(link.attrs['href'])
    except AttributeError as e:
        return None
    return flist



odir = 'R:/UnitedStates/ForDRI_project-NEW/Case_Study_2022_Growing_Season/SM/'
for i in range(1,2): # on the day downloaded the dataset, it has been updated to 253
    if i < 10:
        src = 'https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_NOAH0125_H.002/2022/00'+str(i)+'/'
    elif i < 100:
        src = 'https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_NOAH0125_H.002/2022/0'+str(i)+'/'
    else:
        src = 'https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_NOAH0125_H.002/2022/'+str(i)+'/'
    

    flist = getFile(src)
    if flist == None:
        print("Title could not be found")
    else:
        #print(flist)
        for f in range(len(flist)):
            if flist[f][-3:]=='grb' or flist[f][-3:]=='xml':
                html = src+flist[f]
                # Create an OpenerDirector with support for Basic HTTP Authentication...
                auth_handler = urllib.request.HTTPBasicAuthHandler()
                auth_handler.add_password(realm='NASA Earthdata ',
                                        uri=html,
                                        user='zbc123a',
                                        passwd='NSO5804246tj')
                opener = urllib.request.build_opener(auth_handler)
                # ...and install it globally so it can be used with urlopen.
                urllib.request.install_opener(opener)
                r = requests.get(html, stream = True)
                with open((odir+flist[f].split('/')[-1]), 'wb') as f:
                    f.write(r.content)
                print(html)
            else:
                continue

                

In [55]:
import shutil
html = "https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_NOAH0125_H.002/2022/014/NLDAS_NOAH0125_H.A20220114.0000.002.grb"

user='zbc123a'
password='NSO5804246tj'
session = requests.Session()
session.auth = (user, password)
auth = session.post(html)
r = session.get(html,stream = True)

with open((html.split('/')[-1]), 'wb') as f:
    shutil.copyfileobj(r.raw, f)
f.close()
print(html)

https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_NOAH0125_H.002/2022/014/NLDAS_NOAH0125_H.A20220114.0000.002.grb


# Extract the top soil moisture from the data

In [26]:
#soil mositure content: SOILM Band 25 - 29
#B25: 0-100-DBLY cm
#B26: 0-10-DBLY cm
#B27: 10-40-DBLY cm
#B28: 40-100-DBLY cm
#B29: 100-200-DBLY cm

In [ ]:
# aggregating the data into weekly 2020
sm_dir = "R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\SM_2018_2020\\"
smlist = glob.glob(sm_dir+"2020\\*.tif")
smlist.sort()
for i in range(52):
    no_week=i+1
    weekly_arr = 0
    weekly_smlist = smlist[i*7*24:(i+1)*7*24]
    weekly_smlist.sort()
    for sm in weekly_smlist:
        ds = gdal.Open(sm)
        arr = ds.ReadAsArray()
        arr[arr==9999] = np.nan
        weekly_arr = weekly_arr+arr
    weekly_avg = weekly_arr/(7*24)
    if no_week < 10:
        no_week_str = '0'+str(no_week)
    else:
        no_week_str = str(no_week)
    fname = 'SM_2020'+no_week_str+".tif"
    driver = gdal.GetDriverByName("GTiff")
    TypeIF = gdal.GDT_Float32
    geot = ds.GetGeoTransform()
    proj = ds.GetProjection()
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    Raster = driver.Create(sm_dir + fname, cols, rows, 1, TypeIF)
    Raster.GetRasterBand(1).WriteArray(weekly_avg)
    Raster.GetRasterBand(1).SetNoDataValue(np.nan)
    Raster.SetGeoTransform(geot)
    Raster.SetProjection(proj)
    Raster.FlushCache()
    print(fname) 

In [5]:
#SM2022 growing season
dir_sm = 'R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\SM\\'
odir = 'R:/UnitedStates/ForDRI_project-NEW/Case_Study_2022_Growing_Season/SM_tif/'
flist_sm = glob.glob(dir_sm+'*.grb')
flist_sm.sort()
name_list = ['SM_000_100','SM_000_010','SM_010_040','SM_040_100','SM_100_200']
driver = gdal.GetDriverByName("GTiff")
TypeIF = gdal.GDT_Float32
for f in flist_sm[1320:]:
    ds = gdal.Open(f)
    geot = ds.GetGeoTransform()
    proj = ds.GetProjection()
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    time = os.path.basename(f).split('.')[1][1:]+os.path.basename(f).split('.')[2]
    for depth in range(25,30):
        new_name = name_list[depth-25]+'_'+time+'.tif'
        ds_bands = ds.GetRasterBand(depth)
        df_bands_arr = ds_bands.ReadAsArray()
        Raster = driver.Create(odir + new_name, cols, rows, 1, TypeIF)
        Raster.GetRasterBand(1).WriteArray(df_bands_arr)
        Raster.GetRasterBand(1).SetNoDataValue(9999)
        Raster.SetGeoTransform(geot)
        Raster.SetProjection(proj)
        Raster.FlushCache()
        print(new_name)

SM_000_100_202202250000.tif
SM_000_010_202202250000.tif
SM_010_040_202202250000.tif
SM_040_100_202202250000.tif
SM_100_200_202202250000.tif
SM_000_100_202202250100.tif
SM_000_010_202202250100.tif
SM_010_040_202202250100.tif
SM_040_100_202202250100.tif
SM_100_200_202202250100.tif
SM_000_100_202202250200.tif
SM_000_010_202202250200.tif
SM_010_040_202202250200.tif
SM_040_100_202202250200.tif
SM_100_200_202202250200.tif
SM_000_100_202202250300.tif
SM_000_010_202202250300.tif
SM_010_040_202202250300.tif
SM_040_100_202202250300.tif
SM_100_200_202202250300.tif
SM_000_100_202202250400.tif
SM_000_010_202202250400.tif
SM_010_040_202202250400.tif
SM_040_100_202202250400.tif
SM_100_200_202202250400.tif
SM_000_100_202202250500.tif
SM_000_010_202202250500.tif
SM_010_040_202202250500.tif
SM_040_100_202202250500.tif
SM_100_200_202202250500.tif
SM_000_100_202202250600.tif
SM_000_010_202202250600.tif
SM_010_040_202202250600.tif
SM_040_100_202202250600.tif
SM_100_200_202202250600.tif
SM_000_100_202202250

In [ ]:
# aggregating the data into weekly level
sm_dir = 'R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\SM_tif\\'
smlist = glob.glob(sm_dir+"SM_000_010*.tif")
smlist.sort()

for i in range(35):
    no_week=i+1
    weekly_arr = 0
    weekly_smlist = smlist[i*7*24:(i+1)*7*24]
    weekly_smlist.sort()
    for sm in weekly_smlist:
        ds = gdal.Open(sm)
        arr = ds.ReadAsArray()
        arr[arr==9999] = np.nan
        weekly_arr = weekly_arr+arr
    weekly_avg = weekly_arr/(7*24)
    if no_week < 10:
        no_week_str = '0'+str(no_week)
    else:
        no_week_str = str(no_week)
    fname = 'SM_2022'+no_week_str+".tif"
    driver = gdal.GetDriverByName("GTiff")
    TypeIF = gdal.GDT_Float32
    geot = geoTransform
    proj = projection
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    Raster = driver.Create(sm_dir + fname, cols, rows, 1, TypeIF)
    Raster.GetRasterBand(1).WriteArray(weekly_avg)
    Raster.GetRasterBand(1).SetNoDataValue(np.nan)
    Raster.SetGeoTransform(geot)
    Raster.SetProjection(proj)
    Raster.FlushCache()
    print(fname)
del Raster

## Processing the SM from 1980-2020

In [3]:
# rescale the SM from 1980 to 2017

# check the data scale for the historical data
historical_sm = gdal.Open("R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\nonStanderdizedSM_all\\SM198001.tif").ReadAsArray()
new_sm = gdal.Open("R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\SM_raw_2018_2019_2020\\SM_201801.tif").ReadAsArray()
print("The range of the old sm data (1980-2017) is: %.2f, %.2f" %(np.nanmin(historical_sm), np.nanmax(historical_sm))) #(an example)
print("The range of the new sm data (2018-2020) is: %.2f, %.2f" %(np.nanmin(new_sm), np.nanmax(new_sm))) #(an example)

# rescale the old sm data by deviding by 10
historical_sm_list = glob.glob("R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\nonStanderdizedSM_all\\"+"*.tif")
historical_sm_list.sort()
historical_sm_list = historical_sm_list[:-156]

# new directory to save the dataset
odir_sm = "R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\scaled_originalSM_0_10cm_1980_2020\\"
for f in historical_sm_list:
    ds = gdal.Open(f)
    fname = os.path.basename(f)[2:]
    fname = 'SM_' + fname
    driver = gdal.GetDriverByName("GTiff")
    TypeIF = gdal.GDT_Float32
    geot = ds.GetGeoTransform()
    proj = ds.GetProjection()
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    Raster = driver.Create(odir_sm + fname, cols, rows, 1, TypeIF)
    Raster.GetRasterBand(1).WriteArray(ds.ReadAsArray()/10)
    Raster.GetRasterBand(1).SetNoDataValue(np.nan)
    Raster.SetGeoTransform(geot)
    Raster.SetProjection(proj)
    Raster.FlushCache()
    del Raster
    
new = gdal.Open("R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\scaled_originalSM_0_10cm_1980_2020\\SM_198001.tif").ReadAsArray()
print("The range of the scaled old sm data (1980-2017) is: %.2f, %.2f" %(np.nanmin(new), np.nanmax(new))) #(an example)


The range of the old sm data (1980-2017) is: 35.70, 381.55
The range of the new sm data (2018-2020) is: 3.70, 47.60
The range of the scaled old sm data (1980-2017) is: 3.57, 38.16


In [ ]:
# calculate the mean and std based on the SM from 1980 to 2020
# which will be used in the further normalization for 2022 growing season

sm_dir = "R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/scaled_originalSM_0_10cm_1980_2020/"
smlist = glob.glob(sm_dir+"*.tif")
smlist.sort()

# read the projection from the old sm file
sm_fdir = "R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/scaled_originalSM_0_10cm_1980_2020/SM_201711.tif"
ds = gdal.Open(sm_fdir,GA_ReadOnly)
proj = ds.GetProjectionRef()
geot = ds.GetGeoTransform()

sm_arr = []
for sm in smlist:
    ds = gdal.Open(sm)
    arr = ds.ReadAsArray()
    arr[arr<-2] = np.nan
    sm_arr.append(arr)
sm_arr = np.asarray(sm_arr)

driver = gdal.GetDriverByName("GTiff")
for i in range(1,53):
    if i < 10:
        week = '0'+str(i)
    else:
        week = str(i)
    sm_month = sm_arr[i-1::52,:,:]
    sm_mean = np.mean(sm_month,0)
    mean = driver.Create('R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/SM_MEAN/MEAN_' + week +'.tif', sm_mean.shape[1], sm_mean.shape[0], 1, gdal.GDT_Float32)
    mean.GetRasterBand(1).WriteArray(sm_mean.astype(float))
    sm_std = np.std(sm_month,0)
    std = driver.Create('R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/SM_STD/STD_' + week +'.tif', sm_std.shape[1], sm_std.shape[0], 1, gdal.GDT_Float32)
    std.GetRasterBand(1).WriteArray(sm_std.astype(float))
    print(week)
    mean.FlushCache()
    std.FlushCache()
    mean.SetGeoTransform(geot)
    mean.SetProjection(proj)
    std.SetGeoTransform(geot)
    std.SetProjection(proj)
    del sm_mean, sm_std, mean, std


In [10]:
sm_list = glob.glob('R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/scaled_originalSM_0_10cm_1980_2020/' + '*.tif')
sm_list.sort()
mean_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\SM_MEAN\\' + '*.tif')
mean_list.sort()
std_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\SM_STD\\' + '*.tif')
std_list.sort()

In [11]:
driver = gdal.GetDriverByName("GTiff")
for i in range(52):
    if i+1 < 10:
        week = '0'+str(i+1)
    else:
        week = str(i+1)
    sm_mean = gdal.Open(mean_list[i])
    sm_mean_arr = sm_mean.ReadAsArray()
    sm_std = gdal.Open(std_list[i])
    sm_std_arr = sm_std.ReadAsArray()
    for j in range(41):
        sm = gdal.Open(sm_list[52*j+i])
        sm_arr = sm.ReadAsArray()
        projection = sm.GetProjectionRef()
        geoTransform = sm.GetGeoTransform()
        sm_zscore = (sm_arr-sm_mean_arr)/sm_std_arr
        year = str(1980+j)
        zscore = driver.Create('R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/stdSM_notcropped_original/SM_' + year+week+'.tif', sm_arr.shape[1], sm_arr.shape[0], 1, gdal.GDT_Float32)
        zscore.GetRasterBand(1).WriteArray(sm_zscore)
        zscore.GetRasterBand(1).SetNoDataValue(np.nan)
        zscore.SetGeoTransform(geoTransform)
        zscore.SetProjection(projection)
        zscore.FlushCache()
        del zscore
        print('SM_' + year+week)

SM_198001
SM_198101
SM_198201
SM_198301
SM_198401
SM_198501
SM_198601
SM_198701
SM_198801
SM_198901
SM_199001
SM_199101
SM_199201
SM_199301
SM_199401
SM_199501
SM_199601
SM_199701
SM_199801
SM_199901
SM_200001
SM_200101
SM_200201
SM_200301
SM_200401
SM_200501
SM_200601
SM_200701
SM_200801
SM_200901
SM_201001
SM_201101
SM_201201
SM_201301
SM_201401
SM_201501
SM_201601
SM_201701
SM_201801
SM_201901
SM_202001
SM_198002
SM_198102
SM_198202
SM_198302
SM_198402
SM_198502
SM_198602
SM_198702
SM_198802
SM_198902
SM_199002
SM_199102
SM_199202
SM_199302
SM_199402
SM_199502
SM_199602
SM_199702
SM_199802
SM_199902
SM_200002
SM_200102
SM_200202
SM_200302
SM_200402
SM_200502
SM_200602
SM_200702
SM_200802
SM_200902
SM_201002
SM_201102
SM_201202
SM_201302
SM_201402
SM_201502
SM_201602
SM_201702
SM_201802
SM_201902
SM_202002
SM_198003
SM_198103
SM_198203
SM_198303
SM_198403
SM_198503
SM_198603
SM_198703
SM_198803
SM_198903
SM_199003
SM_199103
SM_199203
SM_199303
SM_199403
SM_199503
SM_199603
SM_199703


In [18]:
# Mask the data into the correct size
out_dir = "R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/stdSM_cropped/"
sm_list = glob.glob('R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/stdSM_notcropped_original/'+'*.tif')
sm_list.sort()
for idx, f in enumerate(sm_list):
    fname = os.path.basename(f)
    time = fname[-10:-4]
    output = out_dir+fname #output file
    data=gdal.Open(f, GA_ReadOnly) #Your data the one you want to clip
    gdal.Translate(output,data,format='GTiff',projWin=[-125.0, 50.0, -67.0, 24.0],outputSRS=projection)

In [20]:
# check the consistency of the data
smlist = glob.glob(out_dir+"*.tif")
sm_arr = []
for sm in smlist:
    ds = gdal.Open(sm)
    arr = ds.ReadAsArray()
    arr[arr<-2] = np.nan
    sm_arr.append(arr)
sm_arr = np.asarray(sm_arr)
print('The mean of the sm from 1980 to 2020 is %.2f, and the std is %.2f (after the normalization)'%(np.nanmean(sm_arr),np.nanstd(sm_arr)))

The mean of the sm from 1980 to 2020 is 0.07, and the std is 0.91 (after the normalization)


In [22]:
# check the consistency of the data
smlist = glob.glob("R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/scaled_originalSM_0_10cm_1980_2020/"+"*.tif")
sm_arr = []
for sm in smlist:
    ds = gdal.Open(sm)
    arr = ds.ReadAsArray()
    arr[arr<-2] = np.nan
    sm_arr.append(arr)
sm_arr = np.asarray(sm_arr)
print('The mean of the sm from 1980 to 2020 is %.2f, and the std is %.2f, the min and max are %.2f and %.2f\
      (before the normalization)'%(np.nanmean(sm_arr),np.nanstd(sm_arr),np.nanmin(sm_arr),np.nanmax(sm_arr)))

The mean of the sm from 1980 to 2020 is 23.88, and the std is 6.42, the min and max are 2.00 and 62.43      (before the normalization)


## SM in the growing season in 2022

In [2]:
ds = gdal.Open('R:/UnitedStates/ForDRI_project-NEW/New_Model_2022/Processing/Soil_Moisture/stdSM_cropped/SM_198001.tif',GA_ReadOnly)# your mask raster
proj = ds.GetProjectionRef()
geot = ds.GetGeoTransform()
minx = geot[0]
maxy = geot[3]
maxx = minx + geot[1] * ds.RasterXSize
miny = maxy + geot[5] * ds.RasterYSize

In [7]:
sm_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\SM_tif\\' + 'SM_2022*.tif')
sm_list.sort()
mean_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\SM_MEAN\\' + '*.tif')
mean_list.sort()
std_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\Soil_Moisture\\SM_STD\\' + '*.tif')
std_list.sort()

In [10]:
driver = gdal.GetDriverByName("GTiff")
for i in range(35): # end in 35
    if i+1 < 10:
        week = '0'+str(i+1)
    else:
        week = str(i+1)
    sm_mean = gdal.Open(mean_list[i])
    sm_mean_arr = sm_mean.ReadAsArray()
    sm_std = gdal.Open(std_list[i])
    sm_std_arr = sm_std.ReadAsArray()
    sm = gdal.Open(sm_list[i])
    sm_arr = sm.ReadAsArray()
    projection = sm.GetProjectionRef()
    geoTransform = sm.GetGeoTransform()
    sm_zscore = (sm_arr-sm_mean_arr)/sm_std_arr
    zscore = driver.Create('R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\stdSM\\SM_2022'+week+'.tif', sm_arr.shape[1], sm_arr.shape[0], 1, gdal.GDT_Float32)
    zscore.GetRasterBand(1).WriteArray(sm_zscore)
    zscore.GetRasterBand(1).SetNoDataValue(np.nan)
    zscore.SetGeoTransform(geoTransform)
    zscore.SetProjection(projection)
    zscore.FlushCache()
    del zscore
    print('SM_2022'+week)

SM_202201
SM_202202
SM_202203
SM_202204
SM_202205
SM_202206
SM_202207
SM_202208
SM_202209
SM_202210
SM_202211
SM_202212
SM_202213
SM_202214
SM_202215
SM_202216
SM_202217
SM_202218
SM_202219
SM_202220
SM_202221
SM_202222
SM_202223
SM_202224
SM_202225
SM_202226
SM_202227
SM_202228
SM_202229
SM_202230
SM_202231
SM_202232
SM_202233
SM_202234
SM_202235


In [11]:
sm_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\stdSM\\' + '*.tif')
sm_list.sort()

In [15]:
out_dir = "R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\stdSM_Masked\\"
for idx, f in enumerate(sm_list):
    fname = os.path.basename(f)
    time = fname[-10:-4]
    output = out_dir+fname #output file
    data=gdal.Open(f, GA_ReadOnly) #Your data the one you want to clip
    gdal.Translate(output,data,format='GTiff',projWin=[-125.0, 50.0, -67.0, 24.0],outputSRS=projection)

In [6]:
# SM projWin
[minx,maxy,maxx,miny]

[-125.0, 50.0, -67.0, 24.0]

In [3]:
# SM proj
proj

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [5]:
# SM geot
geot

(-125.0, 0.125, 0.0, 50.0, 0.0, -0.125)